# DASH VERSION 2 COMPLIQUER

In [1]:
import pandas as pd
from dash import Dash, dash_table, dcc, html, Input, Output
import plotly.express as px

# Charger le fichier CSV
df = pd.read_csv("gapminder_unfiltered.csv")

# Initialiser l'application Dash
app = Dash(__name__)

# Définir la mise en page
app.layout = html.Div(
    style={"padding": "20px", "font-family": "Arial"},
    children=[
        html.H1(
           'Filtre Table et Graph', style={
        'color': 'black',
        'textAlign': 'center',
        'background-color': 'lightgray',
        'padding': '10px',
        'border-radius': '10px'
            },
        ),
        html.Div(
            style={"display": "flex", "gap": "20px"},
            children=[
                # Tableau interactif avec filtre
                html.Div(
                    style={
                        "flex": "1",
                        "border": "1px solid #ddd",
                        "padding": "10px",
                        "border-radius": "5px",
                        "background-color": "#fff",
                        "box-shadow": "0 4px 6px rgba(0, 0, 0, 0.1)",
                    },
                    children=[
                        html.Label(
                            "Filtrer les données :",
                            style={"font-weight": "bold", "margin-bottom": "10px"},
                        ),
                        dcc.Input(
                            id="filter-input",
                            type="text",
                            placeholder="Rechercher (par pays, continent...)",
                            debounce=True,  # Met à jour le tableau après arrêt de la saisie
                            style={
                                "width": "80%",
                                "padding": "10px",
                                "margin-bottom": "10px",
                                "border": "1px solid #ccc",
                                "border-radius": "5px",
                            },
                        ),
                        dash_table.DataTable(
                            id="data-table",
                            columns=[
                                {"name": col, "id": col} for col in df.columns
                            ],
                            data=df.to_dict("records"),
                            page_size=10,
                            style_table={"overflowX": "auto"},
                            style_cell={
                                "textAlign": "left",
                                "padding": "10px",
                                "border": "1px solid #ddd",
                            },
                            style_header={
                                "backgroundColor": "#f4f4f4",
                                "fontWeight": "bold",
                            },
                        ),
                    ],
                ),
                # Graphique interactif
                html.Div(
                    style={
                        "flex": "1",
                        "border": "1px solid #ddd",
                        "padding": "10px",
                        "border-radius": "5px",
                        "background-color": "#fff",
                        "box-shadow": "0 4px 6px rgba(0, 0, 0, 0.1)",
                    },
                    children=[
                        dcc.Dropdown(
                            id="country-dropdown",
                            options=[
                                {"label": country, "value": country}
                                for country in df["country"].unique()
                            ],
                            value="Canada",
                            style={"margin-bottom": "20px"},
                        ),
                        dcc.Graph(id="line-chart"),
                    ],
                ),
            ],
        ),
    ],
)


# Callback pour mettre à jour le tableau en fonction du filtre
@app.callback(
    Output("data-table", "data"),
    [Input("filter-input", "value")],
)
def filter_table(filter_value):
    if not filter_value:
        # Si aucun filtre n'est saisi, afficher toutes les données
        return df.to_dict("records")
    # Filtrer les données en fonction de la saisie (insensible à la casse)
    filtered_df = df[
        df.apply(
            lambda row: row.astype(str)
            .str.contains(filter_value, case=False, na=False)
            .any(),
            axis=1,
        )
    ]
    return filtered_df.to_dict("records")


# Callback pour mettre à jour le graphique
@app.callback(
    Output("line-chart", "figure"),
    [Input("country-dropdown", "value")],
)
def update_chart(selected_country):
    filtered_df = df[df["country"] == selected_country]
    fig = px.line(
        filtered_df,
        x="year",
        y="pop",
        title=f"Population de {selected_country} dans le temps",
    )
    fig.update_layout(
        plot_bgcolor="#f9f9f9",
        paper_bgcolor="#f4f4f9",
        title={"x": 0.5},
        xaxis_title="Année",
        yaxis_title="Population",
    )
    return fig


# Lancer l'application
if __name__ == "__main__":
    app.run_server(debug=True, port=8060)


In [2]:
df

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
3308,Zimbabwe,Africa,1987,62.351,9216418,706.157306
3309,Zimbabwe,Africa,1992,60.377,10704340,693.420786
3310,Zimbabwe,Africa,1997,46.809,11404948,792.449960
3311,Zimbabwe,Africa,2002,39.989,11926563,672.038623
